Install Kaggle Library

In [ ]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 7.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=b076728af268cd4ad705cef67660a114b0ab57ecdd84dafd17ae93e5a6c7e203
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [ ]:
# Upload kaggle API key file
uploaded = files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets list

Saving kaggle.json to kaggle.json


Download and Unzip the data

In [ ]:
!kaggle competitions download -c alaska2-image-steganalysis

100% 30.0G/30.0G [08:36<00:00, 65.3MB/s]
100% 30.0G/30.0G [08:36<00:00, 62.4MB/s]


In [ ]:
!unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: UERD/74676.jpg          
  inflating: UERD/74677.jpg          
  inflating: UERD/74678.jpg          
  inflating: UERD/74680.jpg          
  inflating: UERD/74681.jpg          
  inflating: UERD/74682.jpg          
  inflating: UERD/74683.jpg          
  inflating: UERD/74684.jpg          
  inflating: UERD/74685.jpg          
  inflating: UERD/74686.jpg          
  inflating: UERD/74687.jpg          
  inflating: UERD/74688.jpg          
  inflating: UERD/74689.jpg          
  inflating: UERD/74690.jpg          
  inflating: UERD/74691.jpg          
  inflating: UERD/74692.jpg          
  inflating: UERD/74693.jpg          
  inflating: UERD/74694.jpg          
  inflating: UERD/74695.jpg          
  inflating: UERD/74696.jpg          
  inflating: UERD/74697.jpg          
  inflating: UERD/74698.jpg          
  inflating: UERD/74699.jpg          
  inflating: UERD/74700.jpg          
  inflating: UERD/74701.jpg          

Create the model

In [ ]:
import os
import torch
import numpy as np
import torch.nn.functional as F

from torch.optim import Adam
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, AvgPool2d, Module, Softmax, BatchNorm2d, Dropout, AdaptiveAvgPool2d

def seed_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything()

class AlaskaNet(Module):
  def __init__(self):
    super(AlaskaNet, self).__init__()

    self.inputs = 1 
    self.outputs = 64
    self.kernel_size = (3,3)
    self.dropout_rate = 0

    self.lt1 = Sequential(
      Conv2d(self.inputs, self.outputs, self.kernel_size),
      BatchNorm2d(self.outputs),
      ReLU(),
      Dropout(self.dropout_rate)
    )

    self.lt2 = Sequential(
      #self.lt1(self.inputs, self.outputs, self.kernel_size, self.dropout_rate),
      Conv2d(self.inputs, self.outputs, self.kernel_size),
      BatchNorm2d(self.outputs),
      Dropout(self.dropout_rate)
    )

    self.lt3_branch1 = Sequential(
      #self.lt1(self.inputs, self.outputs, self.kernel_size, self.dropout_rate),
      Conv2d(self.inputs, self.outputs, self.kernel_size),
      BatchNorm2d(self.outputs),
      AvgPool2d(self.kernel_size, stride=2),
      Dropout(self.dropout_rate)
    )

    self.lt3branch2 = Sequential(
      Conv2d(self.inputs, self.outputs, self.kernel_size, stride=2),
      BatchNorm2d(self.outputs),
      Dropout(self.dropout_rate)
    )

    self.lt4 = Sequential(
      #self.lt1(self.inputs, self.outputs, self.kernel_size, self.dropout_rate),
      Conv2d(self.inputs, self.outputs, self.kernel_size),
      BatchNorm2d(self.outputs),
      Dropout(self.dropout_rate),
      AdaptiveAvgPool2d(1)
    )

  def forward(self, x):
    self.inputs = 1 
    self.outputs = 64
    x = self.lt1(x)

    self.inputs = 64 
    self.outputs = 16
    x1 = self.lt1(x)

    self.inputs = 16 
    self.outputs = 16
    x2_1 = self.lt1(x1)
    x2_1 = self.lt2(x2_1)
    x2 = add(x1, x2_1)
    x2_2 = self.lt1(x2)
    x2_2 = self.lt2(x_2)
    x2 = add(x2, x2_2)
    x2_3 = self.lt1(x2)
    x2_3 = self.lt2(x2_3)
    x2 = add(x2, x2_3)
    x2_4 = self.lt1(x2)
    x2_4 = self.lt2(x2_4)
    x2 = add(x2, x2_4)
    x2_5 = self.lt1(x2)
    x2_5 = self.lt2(x2_5)
    x2 = add(x2, x2_5)

    self.inputs = 16 
    self.outputs = 16
    x3_1_1 = self.lt1(x2)
    x3_1_1 = self.lt3_branch1(x3_1_1)
    x3_1_2 = self.lt3_branch2(x2)
    x3 = add(x3_1_1, x3_1_2)

    self.inputs = 16 
    self.outputs = 64
    x3_2_1 = self.lt1(x3)
    x3_2_1 = self.lt3_branch1(x3_2_1)
    x3_2_2 = self.lt3_branch2(x3)
    x3 = add(x3_2_1, x3_2_2)

    self.inputs = 64 
    self.outputs = 128
    x3_3_1 = self.lt1(x3)
    x3_3_1 = self.lt3_branch1(x3_3_1)
    x3_3_2 = self.lt3_branch2(x3)
    x3 = add(x3_3_1, x3_3_2)

    self.inputs = 128 
    self.outputs = 256
    x3_4_1 = self.lt1(x3)
    x3_4_1 = self.lt3_branch1(x3_4_1)
    x3_4_2 = self.lt3_branch2(x3)
    x3 = add(x3_4_1, x3_4_2)


    self.inputs = 256
    self.outputs = 512
    x4 = self.lt1(x3)
    x4 = self.lt4(x4)

    x = Linear(x4)
    x = Softmax(x)


Getting the Quality Factor

In [ ]:
! git clone https://github.com/dwgoon/jpegio
!pip install jpegio/.

import numpy as np
import jpegio as jpio

Cloning into 'jpegio'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 1229 (delta 33), reused 41 (delta 11), pack-reused 1154
Receiving objects: 100% (1229/1229), 231.39 MiB | 33.51 MiB/s, done.
Resolving deltas: 100% (608/608), done.
Processing ./jpegio
  Created wheel for jpegio: filename=jpegio-0.2.1-cp36-cp36m-linux_x86_64.whl size=926869 sha256=9dd18853d8782cfd3e08803d5172795ea501d7448ba02ad55ccc0ac43c56765b
  Stored in directory: /tmp/pip-ephem-wheel-cache-777pls17/wheels/43/06/35/9790194361cd6ba107f79946b97944ed866d359f908a30d21f
Successfully built jpegio


In [ ]:
def JPEGdecompressYCbCr(jpegStruct):
    
    nb_colors=len(jpegStruct.coef_arrays)
        
    [Col,Row] = np.meshgrid( range(8) , range(8) )
    T = 0.5 * np.cos(np.pi * (2*Col + 1) * Row / (2 * 8))
    T[0,:] = T[0,:] / np.sqrt(2)
    
    sz = np.array(jpegStruct.coef_arrays[0].shape)
    
    imDecompressYCbCr = np.zeros([sz[0], sz[1], nb_colors]);
    szDct = (sz/8).astype('int')
    
    
    
    for ColorChannel in range(nb_colors):
        tmpPixels = np.zeros(sz)
    
        DCTcoefs = jpegStruct.coef_arrays[ColorChannel];
        if ColorChannel==0:
            QM = jpegStruct.quant_tables[ColorChannel];
        else:
            QM = jpegStruct.quant_tables[1];
        
        for idxRow in range(szDct[0]):
            for idxCol in range(szDct[1]):
                D = DCTcoefs[idxRow*8:(idxRow+1)*8 , idxCol*8:(idxCol+1)*8]
                tmpPixels[idxRow*8:(idxRow+1)*8 , idxCol*8:(idxCol+1)*8] = np.dot( np.transpose(T) , np.dot( QM * D , T ) )
        imDecompressYCbCr[:,:,ColorChannel] = tmpPixels;
    return imDecompressYCbCr

Splitting Dataset

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

col_names =  ['ImageFileName', 'Label', 'Stego', 'QF']
df  = pd.DataFrame(columns = col_names)

folder_names = ['Cover', 'JMiPOD', 'JUNIWARD', 'UERD']

for dir in folder_names:
  for file in glob.glob(dir+"/*.jpg"):
    
    jpegStruct = jpio.read(file)
    imDecompressYCbCr = JPEGdecompressYCbCr(jpegStruct)

    if (jpegStruct.quant_tables[0][0,0]==2):
        print('Quality Factor is 95')
        qf = 95
    elif (jpegStruct.quant_tables[0][0,0]==3):
        print('Quality Factor is 90')
        qf = 90
    elif (jpegStruct.quant_tables[0][0,0]==8):
        print('Quality Factor is 75')
        qf = 75
    
    print(file.replace('/','_'))
    if dir == 'Cover':
      label = 0
    elif dir == 'JMiPOD':
      if qf == 95:
        label = 1
      elif qf == 90:
        label = 2
      elif qf == 75:
        label = 3
    elif dir == 'JUNIWARD':
      if qf == 95:
        label = 4
      elif qf == 90:
        label = 5
      elif qf == 75:
        label = 6
    elif dir == 'UERD':
      if qf == 95:
        label = 7
      elif qf == 90:
        label = 8
      elif qf == 75:
        label = 9

    new_row = {'ImageFileName':file.replace('/',''), 'Label':label, 'Stego': dir, 'QF':qf}
    df = df.append(new_row, ignore_index=True)

df

# save the combined df 
df.to_csv('df.csv', index=False)

Streaming output truncated to the last 5000 lines.
Cover13098.jpg
Quality Factor is 75
Cover70720.jpg
Quality Factor is 95
Cover79374.jpg
Quality Factor is 90
Cover22130.jpg
Quality Factor is 90
Cover62408.jpg
Quality Factor is 75
Cover53700.jpg
Quality Factor is 90
Cover63273.jpg
Quality Factor is 90
Cover15157.jpg
Quality Factor is 90
Cover43411.jpg
Quality Factor is 95
Cover62120.jpg
Quality Factor is 75
Cover49386.jpg
Quality Factor is 75
Cover10226.jpg
Quality Factor is 75
Cover68662.jpg
Quality Factor is 95
Cover26074.jpg
Quality Factor is 90
Cover02839.jpg
Quality Factor is 90
Cover76993.jpg
Quality Factor is 90
Cover19170.jpg
Quality Factor is 75
Cover74305.jpg
Quality Factor is 95
Cover54136.jpg
Quality Factor is 95
Cover42052.jpg
Quality Factor is 95
Cover10694.jpg
Quality Factor is 75
Cover48376.jpg
Quality Factor is 95
Cover26664.jpg
Quality Factor is 95
Cover55234.jpg
Quality Factor is 95
Cover75990.jpg
Quality Factor is 95
Cover72300.jpg
Quality Factor is 95
Cover37315.jp

In [ ]:
for lbl in range(10):
    df_lbl = df[df['Label']==lbl]
    df_qf_tr, df_qf_val_test = train_test_split(df_lbl, test_size=0.3, random_state=1234, stratify=df_lbl['Label'].values)
    df_qf_val, df_qf_test  = train_test_split(df_qf_val_test, test_size=0.2, random_state=1234, stratify=df_qf_val_test['Label'].values)
    print(f'Split for quality factor of {qf}...')
    #print(df_qf_tr['Label'].value_counts())
    #print(df_qf_val['Label'].value_counts())
    #print(df_qf_test['Label'].value_counts())
    print('Shape of train split: ', df_qf_tr.shape)
    print('Shape of valid split: ', df_qf_val.shape)
    print('Shape of val_test split: ', df_qf_test.shape)
    print('*'*35)
    
    #save the splits
    df_qf_tr.to_csv(f'train_split_qf_{qf}.csv', index=False)
    df_qf_val.to_csv(f'valid_split_qf_{qf}.csv', index=False)
    df_qf_test.to_csv(f'test_val_split_qf_{qf}.csv', index=False)

ValueError: ignored

In [ ]:
!ls

sample_data


Define Optimizer and Loss Function

In [ ]:
# defining the model
model = AlaskaNet()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

AlaskaNet(
  (lt1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0, inplace=False)
  )
  (lt2): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0, inplace=False)
  )
  (lt3_branch1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): AvgPool2d(kernel_size=(3, 3), stride=2, padding=0)
    (3): Dropout(p=0, inplace=False)
  )
  (lt3branch2): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0, inplace=False)
  )
  (lt4): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3)

Train the Model

In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(df_qf_tr), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(df_qf_val), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

Start Training here

In [ ]:
# defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

Check Train and Valid Accuracies

In [ ]:
# prediction for training set
with torch.no_grad():
    output = model(train_x.cuda())
    
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(train_y, predictions)

# prediction for validation set
with torch.no_grad():
    output = model(val_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(val_y, predictions)

Predictions for Test set

In [ ]:
# loading test images
test_img = []
for img_name in tqdm(test['id']):
    # defining the image path
    image_path = 'test_ScVgIM0/test/' + str(img_name) + '.png'
    # reading the image
    img = imread(image_path, as_gray=True)
    # normalizing the pixel values
    img /= 255.0
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    test_img.append(img)

# converting the list to numpy array
test_x = np.array(test_img)
test_x.shape

To keep runtime from disconnecting

function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)